In [1]:

import os
import sys
import keras
import numpy as np
import pandas as pd
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from keras.models import Sequential, Model
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from keras.models import Model, load_model
import keras
from keras import layers
from keras import backend as K
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
import seaborn as sns  
import matplotlib.pyplot as plt
import csv
import matplotlib
import math
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import os
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import umap.umap_ as umap
import tensorflow
import umap.plot
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
import datashader as ds
import datashader.transfer_functions as tf
import datashader.bundling as bd
import matplotlib.pyplot as plt
import colorcet
import matplotlib.colors
import matplotlib.cm
import bokeh.plotting as bpl
import bokeh.transform as btr
import holoviews as hv
import holoviews.operation.datashader as hd


Using TensorFlow backend.


ModuleNotFoundError: No module named 'umap'

In [ ]:
#Load paths to datasets 
Path_Isoforms = '/Users/stef_cm/Desktop/DL_Final/subset.tsv' 
Path = '/Users/stef_cm/Desktop/DL_Final/gtex_gene_expression.tsv' 
latent_dim = 16
intermediate_dim = 32
epochz= 2
batch_size = 420


In [ ]:
#Load all the tools we need for our VAE , Plotting and data loading 
def load_data(DATA_PATH):
    """Loads the data and preprocesses it."""
    row_names = []
    array = []
    with open(DATA_PATH, 'r', encoding='utf-8') as infile:
        next(infile)
        for line in infile:
            line = line.split("\t")
            row_names.append(line[0])
            array.append(line[1:])

    expr = np.asarray(array, dtype=np.float32)
    expr = np.log2(1+expr[:])
    expr = expr / np.max(expr, axis=1, keepdims=True)
    expr = np.nan_to_num(expr)
    return expr, row_names


def train(autoencoder, data , epochs=epochz):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        for x in data:
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            loss.backward()
            opt.step()
    return autoencoder

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon


def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

In [ ]:
#Load the data and use the *load data * function 

x, row_names = load_data(Path)
original_dim = x.shape[1]
original_dim

# Transform the inputs to tensors for our VAE and define optimizer for the model
inputs = keras.Input(shape=(original_dim,))
outputs = layers.Dense(original_dim)(x) # No activation
opt = tf.keras.optimizers.Adam(learning_rate=0.00001, clipnorm=0.1)

In [ ]:
#VAE model, with connected layers. We also save the model for future work ect.

class VAE(keras.Model):
    def __init__(self, original_dim=ori_dim, epochs = epochz, intermediate_dim = intermediate_dim, latent_dim = latent_dim):
        super(VAE, self).__init__()
        inputs = keras.Input(shape=(original_dim,))
        h = layers.Dense(intermediate_dim)(inputs) # delete act
        
        z_mean = layers.Dense(latent_dim)(h)
        z_log_sigma = layers.Dense(latent_dim)(h)
        z = layers.Lambda(sampling)([z_mean, z_log_sigma])

        # Create encoder
        encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
        self.encoder = encoder
        # Create decoder
        latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
        x = layers.Dense(intermediate_dim)(latent_inputs) #relu
        
        
        outputs = layers.Dense(original_dim)(x) #delete act
        decoder = keras.Model(latent_inputs, outputs, name='decoder')
        self.decoder = decoder
        
        # instantiate VAE model
        outputs = decoder(encoder(inputs)[2])
        vae = keras.Model(inputs, outputs, name='vae_mlp')
        
        # loss
        reconstruction_loss = keras.losses.mean_squared_error(inputs, outputs) #mean and a varriance
        reconstruction_loss *= original_dim        
        kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5   
        vae_loss = K.mean(0.9*(reconstruction_loss) + 0.1*(kl_loss))
        vae.add_loss(vae_loss)
        
        opt = tf.keras.optimizers.Adam(learning_rate=0.0001 ,clipnorm=0.001)

        vae.compile(optimizer= opt, loss='mean_squared_logarithmic_error', metrics=['accuracy'])
        history = vae.fit(y_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(y_test, y_test))
        vae.save("test_vae2")
        
    

In [ ]:
#Train the model on the Gene_expression dataset
x_train = x_test = np.array(x) 
vae = VAE(intermediate_dim = intermediate_dim, latent_dim = latent_dim, epochs=epochz)

*Plotting and predictions*

In [ ]:
plot_history(history)
# encode
x_test_encoded = np.array(vae.encoder.predict(x_test , batch_size=batch_size))
#Visualizations , sanity check 
mapper = umap.UMAP().fit(y)
umap.plot.points(mapper)
standard_embedding = umap.UMAP(random_state=42).fit_transform(x)
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], s=0.1, cmap='Spectral');


In [ ]:
#Latent space visualization 
# display a 2D plot of the digit classes in the latent space
z_mean = np.array(vae.encoder.predict( outputs , batch_size=120))
plt.figure(figsize=(12, 10))
plt.scatter(z_mean[:, 0:500], z_mean[:, 501:1001],c=z_mean[:0:1001])
plt.xlabel("z[0]")
plt.ylabel("z[1]")
plt.title("Latent space",fontsize=22)
plt.show()

In [ ]:
# Perform PCA on the data
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(df_gene)
y_pca = pca.fit_transform(df_isoform)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)
component_names_1 = [f"PC{i+1}" for i in range(y_pca.shape[1])]
y_pca = pd.DataFrame(y_pca, columns=component_names_1)

X = X_pca # training on
y = y_pca # target

# X_pca.head()
principal_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2'])

# Fit a linear regression model on the transformed data
model = LinearRegression().fit(X, y)
model.fit(X, y)

# Use the model to make predictions
predictions = model.predict(X)

# Figure size
plt.figure(figsize=(8,6))
plt.scatter(principal_df.iloc[:,0], principal_df.iloc[:,1], s=40)
plt.title('PCA plot in 2D')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

print(X_pca.shape, y.shape)

In [ ]:
# We used KMeans to cluster together the tissues so we can get labels for our plots , this method it's not the optimal one but we assume tissues are clustering together. 

# KMeans
kmeans = KMeans(n_clusters=24, n_init=15, max_iter=1000, random_state=69)
# Train and make predictions
clusters = kmeans.fit_predict(predictions)
# Cluster centers
centroids = kmeans.cluster_centers_

um = umap.UMAP()
X_fit = um.fit(predictions)           
X_umap = um.transform(predictions)

# Convert to data frame
umap_df = pd.DataFrame(data = X_umap, columns = ['umap comp. 1', 'umap comp. 2'])

# Shape and preview
print(umap_df.shape)
umap_df.head()

# Figure size
plt.figure(figsize=(8,6))
plt.scatter(umap_df.iloc[:,0], umap_df.iloc[:,1], c=clusters, cmap="brg", s=40)

# Centroids
centroids_umap = um.transform(centroids)
plt.scatter(x=centroids_umap[:,0], y=centroids_umap[:,1], marker="x", s=500, linewidths=3, color="black")

# Aesthetics
plt.title('UMAP plot in 2D')
plt.xlabel('umap component 1')
plt.ylabel('umap component 2')

In [ ]:
# Use UMAP to reduce the dimensionality of the data
umap_model = UMAP(n_components=2)
X_transformed = umap_model.fit_transform(x)

# Split the data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_transformed,  test_size=0.2, random_state=42)

# Initialize the random forest model
model = RandomForestClassifier(n_estimators=100)

# Initialize a list to store the validation losses for each iteration
val_losses = []

# Train the model and calculate the validation loss for each iteration
for i in range(100):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    val_loss = np.mean((y_pred - y_val)**2)
    val_losses.append(val_loss)

# Plot the validation losses
plt.plot(val_losses)
plt.xlabel('Iteration')
plt.ylabel('Validation Loss')
plt.show()


In [ ]:
#Elbow 
distortions = []
K = range(15,50)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X_transformed)
    distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
#using UMAP to visualize the 24 clusters of different tissues.
mapper = umap.UMAP(min_dist=1 , n_neighbors=25).fit(X_transformed)
umap.plot.points(mapper, labels= clusters )